# Halite 

## 

In [1]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../Environment/")
import halite_env as Env

In [2]:
from importlib import reload
reload(Env)


<module 'halite_env' from '../Environment/halite_env.py'>

In [3]:
num_players = 1
map_size = 7 # 7 x 7 map

In [4]:
HEnv = Env.HaliteEnv(num_players, map_size)

In [5]:
def Single_ship_env_initialization(map_size, num_players = 1):
    HEnv = Env.HaliteEnv(num_players, map_size)
    return Env.SingleShipEnv(HEnv, 1, map_size)

def check_position(env):
    print(env.state[:,:,:])


In [6]:
env = Single_ship_env_initialization(map_size)
env.reset()
check_position(env)

[[[147   0   0   0   0]
  [ 10   0   0   0   0]
  [448   0   0   0   0]
  [973   0   0   0   0]
  [883   0   0   0   0]
  [237   0   0   0   0]
  [332   0   0   0   0]]

 [[342   0   0   0   0]
  [834   0   0   0   0]
  [116   0   0   0   0]
  [850   0   0   0   0]
  [222   0   0   0   0]
  [123   0   0   0   0]
  [446   0   0   0   0]]

 [[302   0   0   0   0]
  [188   0   0   0   0]
  [463   0   0   0   0]
  [970   0   0   0   0]
  [483   0   0   0   0]
  [867   0   0   0   0]
  [289   0   0   0   0]]

 [[447   0   0   0   0]
  [956   0   0   0   0]
  [583   0   0   0   0]
  [  0   1   0   1   1]
  [ 51   0   0   0   0]
  [719   0   0   0   0]
  [236   0   0   0   0]]

 [[595   0   0   0   0]
  [292   0   0   0   0]
  [735   0   0   0   0]
  [366   0   0   0   0]
  [ 29   0   0   0   0]
  [243   0   0   0   0]
  [729   0   0   0   0]]

 [[  0   0   0   0   0]
  [250   0   0   0   0]
  [ 81   0   0   0   0]
  [156   0   0   0   0]
  [333   0   0   0   0]
  [457   0   0   0   0]
  [903

In [7]:
env.step(1)
check_position(env)


[[[147   0   0   0   0]
  [ 10   0   0   0   0]
  [448   0   0   0   0]
  [973   0   0   0   0]
  [883   0   0   0   0]
  [237   0   0   0   0]
  [332   0   0   0   0]]

 [[342   0   0   0   0]
  [834   0   0   0   0]
  [116   0   0   0   0]
  [850   0   0   0   0]
  [222   0   0   0   0]
  [123   0   0   0   0]
  [446   0   0   0   0]]

 [[302   0   0   0   0]
  [188   0   0   0   0]
  [463   0   0   0   0]
  [970   0   0   0   0]
  [483   0   0   0   0]
  [867   0   0   0   0]
  [289   0   0   0   0]]

 [[447   0   0   0   0]
  [956   0   0   0   0]
  [583   0   0   0   0]
  [  0   0   0   1   0]
  [ 51   0   0   0   0]
  [719   0   0   0   0]
  [236   0   0   0   0]]

 [[595   0   0   0   0]
  [292   0   0   0   0]
  [735   0   0   0   0]
  [366   1   0   0   1]
  [ 29   0   0   0   0]
  [243   0   0   0   0]
  [729   0   0   0   0]]

 [[  0   0   0   0   0]
  [250   0   0   0   0]
  [ 81   0   0   0   0]
  [156   0   0   0   0]
  [333   0   0   0   0]
  [457   0   0   0   0]
  [903

## Baselines

In [27]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2, ACKTR
import stable_baselines
reload(stable_baselines)

MULTI = True
COMPARISON = False
obs_type = "raw"
n_timesteps = 100000
map_size = 7

In [23]:
def make_env(map_size, rank, seed=1234):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param rank: (int) index of the subprocess
    :param seed: (int) the inital seed for RNG
    """

    def _init():
        env = Single_ship_env_initialization(map_size)
        env.seed(seed + rank)
        return env

    set_global_seeds(seed)
    return _init

def evaluate(model, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward
    """
    episode_rewards = [[0.0] for _ in range(env.num_envs)]
    obs = env.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        actions, _states = model.predict(obs)
        # here, action, rewards and dones are arrays
        # because we are using vectorized env
        obs, rewards, dones, info = env.step(actions)

        # Stats
        for i in range(env.num_envs):
            episode_rewards[i][-1] += rewards[i]
            if dones[i]:
                episode_rewards[i].append(0.0)

    mean_rewards = [0.0 for _ in range(env.num_envs)]
    n_episodes = 0
    for i in range(env.num_envs):
        mean_rewards[i] = np.mean(episode_rewards[i])
        n_episodes += len(episode_rewards[i])

    # Compute mean reward
    mean_reward = round(np.mean(mean_rewards), 1)
    print("Mean reward:", mean_reward, "Num episodes:", n_episodes)

    return mean_reward





if MULTI:
    num_cpu = 32  # Number of processes to use
    # Create the vectorized environment
    env = SubprocVecEnv([make_env(map_size, i) for i in range(num_cpu)])
else:
    # env = SingleSnek(obs_type="rgb", n_food=3)
    env = Single_ship_env_initialization(map_size)
    # The algorithms require a vectorized environment to run
    env = DummyVecEnv([lambda: env])


ob = env.reset()



In [24]:
model = stable_baselines.PPO2(MlpPolicy, env, verbose=1, tensorboard_log="/tmp/halite/")

In [25]:
mean_reward_before_train = evaluate(model, num_steps=1000)

Mean reward: -6.1 Num episodes: 96


In [26]:
if MULTI:
    # Multiprocessed RL Training
    start_time = time.time()
    model.learn(n_timesteps)
    total_time_multi = time.time() - start_time

    print(
        "Took {:.2f}s for multiprocessed version - {:.2f} FPS".format(
            total_time_multi, n_timesteps / total_time_multi
        )
    )
    
    if COMPARISON:
        # Single Process RL Training
        env = Single_ship_env_initialization(map_size)
        env = DummyVecEnv([lambda: env])

        single_process_model = PPO2(MlpPolicy, env, verbose=0)

        start_time = time.time()
        single_process_model.learn(n_timesteps)
        total_time_single = time.time() - start_time

        print(
            "Took {:.2f}s for single process version - {:.2f} FPS".format(
                total_time_single, n_timesteps / total_time_single
            )
        )

        print(
            "Multiprocessed training is {:.2f}x faster!".format(
                total_time_single / total_time_multi
            )
        )

    mean_reward = evaluate(model, num_steps=10000)
else:
    start_time = time.time()
    model.learn(n_timesteps)
    total_time_single = time.time() - start_time

    print(
        "Took {:.2f}s for training - {:.2f} FPS".format(
            total_time_single, n_timesteps / total_time_single
        )
    )

    mean_reward = evaluate(model, num_steps=10000)


--------------------------------------
| approxkl           | 0.00085096736 |
| clipfrac           | 0.0           |
| explained_variance | -1.01         |
| fps                | 2609          |
| nupdates           | 1             |
| policy_entropy     | 1.385457      |
| policy_loss        | -0.00151566   |
| serial_timesteps   | 128           |
| time_elapsed       | 2.15e-06      |
| total_timesteps    | 4112          |
| value_loss         | 0.06463078    |
--------------------------------------
---------------------------------------
| approxkl           | 0.0005664417   |
| clipfrac           | 0.0            |
| explained_variance | -2.15          |
| fps                | 3730           |
| nupdates           | 2              |
| policy_entropy     | 1.3816876      |
| policy_loss        | -0.00090386905 |
| serial_timesteps   | 256            |
| time_elapsed       | 1.57           |
| total_timesteps    | 8224           |
| value_loss         | 0.03241264     |
-------------

--------------------------------------
| approxkl           | 0.0006634569  |
| clipfrac           | 0.0           |
| explained_variance | -0.89         |
| fps                | 3773          |
| nupdates           | 18            |
| policy_entropy     | 1.2433133     |
| policy_loss        | -0.0018714822 |
| serial_timesteps   | 2304          |
| time_elapsed       | 19.6          |
| total_timesteps    | 74016         |
| value_loss         | 0.004517697   |
--------------------------------------
--------------------------------------
| approxkl           | 0.0046506138  |
| clipfrac           | 0.022949219   |
| explained_variance | -0.00152      |
| fps                | 3593          |
| nupdates           | 19            |
| policy_entropy     | 1.1700217     |
| policy_loss        | -0.0032723963 |
| serial_timesteps   | 2432          |
| time_elapsed       | 20.7          |
| total_timesteps    | 78128         |
| value_loss         | 0.06688022    |
-------------------------

In [ ]:
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()


In [ ]:
model.save("halite")
del model
model = PPO2.load("halite", env=env)


In [ ]:
env = Single_ship_env_initialization(map_size)

In [ ]:
env.reset()
for i in range(420):
    ob = env.step(env.action_space.sample())
print(ob)

In [ ]:
env.step(env.action_space.sample())
#env.reset()


In [ ]:
x = gym.spaces.Discrete(4)

In [ ]:
x.sample()

In [ ]:
import time
from random import uniform
from IPython.display import display, clear_output

i = 1
for _ in range(200):
    clear_output(wait=True)
    display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
    time.sleep(0.1)
    i += 1